# Importing the libraries

In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


## GAN Model Synthesizers

In [2]:
def define_gan(g_model, dis_model):
    dis_model.trainable = False
    model = keras.models.Sequential()
    model.add(g_model)
    model.add(dis_model)
    opt = keras.optimizers.adam(learning_rate= 0.0002,
                                beta_1= 0.5)
    
    model.compile(loss= 'binary_crossentropy',
                  optimizer= opt)
    return model

## Discriminator and Generator Model Synthesizers

In [3]:
def define_generator(latent_dim):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(units= 128 * 7 * 7,
                                 input_dim= latent_dim))
    model.add(keras.layers.Reshape((7, 7, 128)))
    
    model.add(keras.layers.Conv2DTranspose(filters= 128,
                                           kernel_size= (4,4),
                                           strides= (2,2),
                                           padding= 'same'))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Conv2DTranspose(filters= 128,
                                           kernel_size= (4,4),
                                           strides= (2,2),
                                           padding= 'same'))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Conv2D(filters= 1,
                                  kernel_size= (7,7),
                                  activation= 'sigmoid',
                                  padding= 'same'))
    return model

In [4]:
def define_discriminator(input_shape= (28,28,1)):
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(filters= 64,
                                  strides= (2,2),
                                  kernel_size= (3, 3),
                                  padding= 'same',
                                  input_shape= input_shape))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Conv2D(filters= 64,
                                  strides= (2,2),
                                  kernel_size= (3, 3),
                                  padding= 'same',
                                  input_shape= input_shape))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(units= 1,
                                 activation= 'sigmoid'))
    opt = keras.optimizers.adam(learning_rate= 0.0002, beta_1= 0.5)
    model.compile(loss= 'binary_crossentropy', optimizer= opt)
    return model

# Compiling the models altogether

In [5]:
latent_dim = 100
g_model = define_generator(latent_dim= latent_dim)
d_model = define_discriminator()
model = define_gan(g_model= g_model,
                   dis_model= d_model)
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_1 (Sequential)    (None, 28, 28, 1)         1164289   
_________________________________________________________________
sequential_2 (Sequential)    (None, 1)                 40705     
Total params: 1,204,994
Trainable params: 1,164,289
Non-trainable params: 40,705
_________________________________________________________________


## Generating Real Exampels

In [12]:
def load_mnist_data():
    (X_train, _), (_, _) = keras.datasets.mnist.load_data()
    X_train = np.expand_dims(X_train, axis= -1).astype('float32') / 255.0
    return X_train

In [13]:
def generate_real_samples(dataset, n_samples):
    ix = np.random.randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    y = np.ones((n_samples, 1))
    return X, y

## Generating Fake Examples

In [6]:
def generate_latent_points(latent_dim, n_samples):
    x_input = np.random.randn(latent_dim * n_samples)
    x_input = x_input.reshape((n_samples, latent_dim))
    return x_input

In [7]:
def generate_fake_samples(g_model, latent_dim, n_samples):
    X = generate_latent_points(latent_dim= latent_dim,
                               n_samples= n_samples)
    y = np.zeros((n_samples, 1))
    return X, y

## GAN Model Trainer

In [15]:
def train_gan(gan_model, g_model, d_model, dataset, latent_dim, iters= 200, batch_size= 256):
    half_batch = int(batch_size / 2)
    batch_per_epoch = dataset.shape[0]/batch_size
    for i in range(iters):
        for j in range(batch_per_epoch):
            X_real, y_real = generate_real_samples(half_batch)
            X_fake, y_fake = generate_fake_samples(half_batch)
            X, y = np.vstack((X_real, X_fake)), np.vstack(y_real, y_fake)
            d_loss, _ = d_model.train_on_batch(X, y)
            
            x_input = generate_latent_points(latent_dim,
                                             batch_size= batch_size)
            X_gan = g_model.predict(x_input)
            y_gan = np.ones((n_samples, 1))
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            if not j%10:
                print(f'Discriminator Loss is {d_loss}, Generator Loss is {g_loss}')